In [22]:
pip install SpeechRecognition pydub

In [7]:
audio_path = "audio.mp3"

In [10]:
from pydub import AudioSegment
from pydub.utils import which

AudioSegment.converter = which("ffmpeg")

# Ruta del archivo de audio
audio_path = "audio/Audio2.mp3"

audio_wav = "audio_temp.wav"
AudioSegment.from_file(audio_path).export(audio_wav, format="wav")


<_io.BufferedRandom name='audio_temp.wav'>

In [11]:
import speech_recognition as sr
from pydub import AudioSegment

# Ruta del archivo de audio (cámbiala por la tuya)
audio_path = "audio/Audio2.mp3"


audio_wav = "audio_temp.wav"
AudioSegment.from_file(audio_path).export(audio_wav, format="wav")


recognizer = sr.Recognizer()


with sr.AudioFile(audio_wav) as source:
    print("Cargando el audio...")
    audio_data = recognizer.record(source)

try:
    transcription = recognizer.recognize_google(audio_data, language="es-ES")
    print("Transcripción:")
    print(transcription)
except sr.UnknownValueError:
    print("No se pudo entender el audio.")
except sr.RequestError as e:
    print(f"Error del servicio de reconocimiento: {e}")


Cargando el audio...
Error del servicio de reconocimiento: recognition request failed: Bad Request


In [12]:
from pydub import AudioSegment

# Cargar el archivo de audio
audio = AudioSegment.from_file("audio_temp.wav")

# Dividir en segmentos de 1 minuto (60,000 ms)
segment_duration = 60 * 1000
for i, segment in enumerate(audio[::segment_duration]):
    segment.export(f"segment_{i}.wav", format="wav")


In [30]:
import os
import speech_recognition as sr

recognizer = sr.Recognizer()

# Directorio donde se encuentran los archivos de audio
audio_directory = "segmentos"  # Cambia esta ruta si es necesario
audio_files = sorted([f for f in os.listdir(audio_directory) if f.endswith(".wav")])

# Archivo para guardar la transcripción
output_file = "transcripcion.txt"

# Transcribir cada archivo
with open(output_file, "w", encoding="utf-8") as file:
    for audio_file in audio_files:
        audio_path = os.path.join(audio_directory, audio_file)
        print(f"Transcribiendo: {audio_file}")

        try:
            with sr.AudioFile(audio_path) as source:
                audio_data = recognizer.record(source)
                transcription = recognizer.recognize_google(audio_data, language="es-ES")
                print(f"Transcripción de {audio_file}: {transcription}\n")

                # Escribir en el archivo de salida
                file.write(f"--- Transcripción de {audio_file} ---\n")
                file.write(transcription + "\n\n")
        except sr.UnknownValueError:
            print(f"No se pudo entender el audio: {audio_file}")
            file.write(f"--- Transcripción de {audio_file} ---\n")
            file.write("No se pudo entender el audio.\n\n")
        except sr.RequestError as e:
            print(f"Error del servicio de reconocimiento: {e}")
            file.write(f"Error del servicio para {audio_file}: {e}\n\n")

print(f"Transcripciones guardadas en {output_file}")


Transcribiendo: segment_0.wav


KeyboardInterrupt: 

In [18]:
import os
import speech_recognition as sr

# Inicializar el reconocedor
recognizer = sr.Recognizer()

# Directorio donde se encuentran los archivos de audio
audio_directory = "segmentos"  # Cambia esta ruta si es necesario
audio_files = sorted([f for f in os.listdir(audio_directory) if f.endswith(".wav")])

# Archivo para guardar la transcripción continua
output_file = "transcripcion_continua.txt"

# Variable para almacenar todas las transcripciones
texto_completo = ""

# Transcribir cada archivo y combinar en un solo texto
for audio_file in audio_files:
    audio_path = os.path.join(audio_directory, audio_file)
    print(f"Transcribiendo: {audio_file}")

    try:
        with sr.AudioFile(audio_path) as source:
            audio_data = recognizer.record(source)
            transcription = recognizer.recognize_google(audio_data, language="es-ES")
            print(f"Transcripción de {audio_file}: {transcription}\n")

            # Agregar al texto completo
            texto_completo += transcription + " "
    except sr.UnknownValueError:
        print(f"No se pudo entender el audio: {audio_file}")
        texto_completo += "[No se pudo entender este fragmento.] "
    except sr.RequestError as e:
        print(f"Error del servicio de reconocimiento: {e}")
        texto_completo += f"[Error del servicio: {e}] "

# Guardar el texto completo en el archivo de salida
with open(output_file, "w", encoding="utf-8") as file:
    file.write(texto_completo)

print(f"Transcripción completa guardada en {output_file}")


Transcribiendo: segment_0.wav
Transcripción de segment_0.wav: una comunicación entre Cali y Windows vale Si no tenemos la red Nat porque podemos tener varias redes más montadas para nuestros equipos por ejemplo en casa por wi-fi no es lo mismo que cuando lo hago por cable os habéis dado cuenta que cuando voy por cable me entregan $122 168.2 y tal en cambio por wi-fi me da otro otro Rango pero normalmente empiezan por 10 entonces podéis tener varias redes Nat sí se entiende la idea Este es el primer problema porque si no no llegaré a destino y no podré analizar y podré ver lo que pasa en la máquina objetivo que en este caso la máquina objetivo estaría preparada dentro de lo que es Cali de acuerdo Entonces qué he hecho primer paso cómo configuro la red

Transcribiendo: segment_1.wav
Transcripción de segment_1.wav: nosotros desde esta opción primero os explico el paso de cómo seríais no aquí en Cali tenéis que olvidaros un poco de la idea de Cali porque quién es el que manda en las comuni

In [29]:
!pip install fpdf
from fpdf import FPDF

# Ruta del archivo de texto y nombre del archivo PDF
input_txt_file = "transcripcion_continua.txt"
output_pdf_file = "transcripcion_continua.pdf"

# Crear un objeto PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

# Leer el contenido del archivo de texto y agregarlo al PDF
with open(input_txt_file, "r", encoding="utf-8") as file:
    for line in file:
        pdf.multi_cell(0, 10, line)

# Guardar el PDF
pdf.output(output_pdf_file)
print(f"Archivo PDF guardado como {output_pdf_file}")


UnicodeEncodeError: 'latin-1' codec can't encode character '\u20ac' in position 1951: ordinal not in range(256)